Text Summarization give summary of paragraph,documents

In [ ]:
## install hugging face tranformers lib. and torch lib.
!pip install transformers==3.5.1
!pip install torch==1.4.0
!pip install wikipedia==1.4.0

##**Text Summarization with BART**

In [3]:
from transformers import BartTokenizer, BartForConditionalGeneration 


In [4]:
# Pre-trained BART model  
Model_Name='facebook/bart-large-cnn'

## https://huggingface.co/docs/transformers/model_doc/bart#transformers.BartForConditionalGeneration
## https://huggingface.co/facebook/bart-large-cnn

model=BartForConditionalGeneration.from_pretrained(Model_Name)
tokenizer=BartTokenizer.from_pretrained(Model_Name)

In [5]:
#text="Pandemics do eventually end, even if omicron is complicating the question of when this one will. But it won't be like flipping a light switch: The world will have to learn to coexist with a virus that's not going away.The ultra-contagious omicron mutant is pushing cases to all-time highs and causing chaos as an exhausted world struggles, again, to stem the spread. But this time, we're not starting from scratch.Vaccines offer strong protection from serious illness, even if they don't always prevent a mild infection. Omicron doesn't appear to be as deadly as some earlier variants. And those who survive it will have some refreshed protection against other forms of the virus that still are circulating — and maybe the next mutant to emerge, too.The newest variant is a warning about what will continue to happen unless we really get serious about the endgame, said Dr. Albert Ko, an infectious disease specialist at the Yale School of Public Health.Certainly COVID will be with us forever, Ko added. We're never going to be able to eradicate or eliminate COVID, so we have to identify our goals.At some point, the World Health Organization will determine when enough countries have tamped down their COVID-19 cases sufficiently — or at least, hospitalizations and deaths — to declare the pandemic officially over. Exactly what that threshold will be isn't clear.Even when that happens, some parts of the world still will struggle — especially low-income countries that lack enough vaccines or treatments — while others more easily transition to what scientists call an endemic state.They're fuzzy distinctions, said infectious disease expert Stephen Kissler of the Harvard T.H. Chan School of Public Health. He defines the endemic period as reaching some sort of acceptable steady state to deal with COVID-19.The omicron crisis shows we're not there yet but I do think we will reach a point where SARS-CoV-2 is endemic much like flu is endemic, he said."
import wikipedia
wiki = wikipedia.page('Artificial Intelligence')
text=wiki.content

In [6]:
tokenize_input=tokenizer(text, max_length=1024, truncation=True,return_tensors="pt")

In [7]:
## Get token IDs of summary
length_of_summary=100
summary_tokenids=model.generate(tokenize_input['input_ids'], num_beams=4,  max_length=length_of_summary, early_stopping=True)

In [8]:
## from summary IDs , get corresponding tokens/words
text_summary = ([tokenizer.decode(i, skip_special_tokens=True, clean_up_tokenization_spaces=False) for i in summary_tokenids])

In [9]:
print(text_summary)

['Artificial intelligence (AI) is intelligence demonstrated by machines, as opposed to natural intelligence displayed by animals including humans. Some popular accounts use the term "artificial intelligence" to describe machines that mimic "cognitive" functions that humans associate with the human mind. The field was founded on the assumption that human intelligence "can be so precisely described that a machine can be made to simulate it"']


##**HuggingFace Pipeline for summarization using T5, BART , Pegasus**

In [12]:
import wikipedia
wiki = wikipedia.page('Artificial Intelligence')
text=wiki.content

In [9]:
from transformers import pipeline
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

model_summaries = {}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


T5 model

In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("t5-large")
model = AutoModelWithLMHead.from_pretrained("t5-large")

pipe = pipeline("summarization", model=model, tokenizer= tokenizer)
##pipe = pipeline("summarization", model="t5-large")


pipe_out = pipe(text)
model_summaries["t5"] = "n".join(sent_tokenize(pipe_out[0]["summary_text"]))

BART model

In [ ]:
pipe = pipeline("summarization", model="facebook/bart-large-cnn")
pipe_out = pipe(text)
model_summaries["bart"] = "n".join(sent_tokenize(pipe_out[0]["summary_text"]))

Pegasus model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-cnn_dailymail")
model = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-cnn_dailymail")

pipe = pipeline("summarization", model=model, tokenizer= tokenizer)
pipe_out = pipe(text)
model_summaries["pegasus"] = pipe_out[0]["summary_text"].replace(" .<n>", ".n")

In [ ]:
for model_name in model_summaries:
    print(model_name.upper())
    print(model_summaries[model_name])
    print("")

##**Document summarization with ktrain**

https://github.com/amaiya/ktrain

In [ ]:
!pip install ktrain==0.25.3
!pip install transformers==4.11.3

In [1]:
import wikipedia

In [2]:
# Specify the title of the Wikipedia page which we want to extract:
wiki = wikipedia.page('Artificial Intelligence')

In [3]:
doc = wiki.content

In [4]:
print(doc[:1000])

Artificial intelligence (AI) is intelligence demonstrated by machines, as opposed to natural intelligence displayed by animals including humans.
Leading AI textbooks define the field as the study of "intelligent agents": any system that perceives its environment and takes actions that maximize its chance of achieving its goals.
Some popular accounts use the term "artificial intelligence" to describe machines that mimic "cognitive" functions that humans associate with the human mind, such as "learning" and "problem solving", however, this definition is rejected by major AI researchers.AI applications include advanced web search engines (e.g., Google), recommendation systems (used by YouTube, Amazon and Netflix), understanding human speech (such as Siri and Alexa), self-driving cars (e.g., Tesla), automated decision-making and competing at the highest level in strategic game systems (such as chess and Go).
As machines become increasingly capable, tasks considered to require "intelligence

In [5]:
from ktrain import text

ts = text.TransformerSummarizer()

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

In [6]:
ts.summarize(doc)

'Artificial intelligence (AI) is intelligence demonstrated by machines, as opposed to natural intelligence displayed by animals including humans. Some popular accounts use the term "artificial intelligence" to describe machines that mimic "cognitive" functions that humans associate with the human mind. The field was founded on the assumption that human intelligence "can be so precisely described that a machine can be made to simulate it"'